In [176]:
import pandas as pd
import numpy as np
import glob
import os
import json
import csv
import math
import scipy.sparse
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from bs4 import BeautifulSoup
import requests
from sklearn.utils import shuffle
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.estimator_checks import check_estimator
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from collections import defaultdict
from sklearn.model_selection import PredefinedSplit

In [177]:
pd.set_option('display.max_colwidth', -1)

Start by exploring academic training dataset.

In [178]:
all_words = pd.read_csv('data/train/academic/words.csv', encoding='ISO--8859-1', na_filter=False)
display(len(all_words))
all_words.head()

48964

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,25070,HER,she,DPS,,,1267,a6u-fragment02,a6u,academic
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic


## Keep only content words and filter stop verbs!

In [179]:
# Keeping content words: http://ucrel.lancs.ac.uk/bnc2/bnc2guide.htm#m2adv
# Does not include forms of 'be', 'do', 'have', or modal verbs,
# as suggested in https://github.com/EducationalTestingService/metaphor/tree/master/content-words
content_word_pos = set([
    'NN1', 'NN2', 'NN0',
    'NP0',
    'NN1-NP0', 'NN1-VVB', 'NN1-VVG', 'NN2-VVZ',
    'NP0-NN1',
    'VVB', 'VVD', 'VVG', 'VVI', 'VVN', 'VVZ',
    'VVB-NN1', 'VVD-VVN', 'VVD-AJ0', 'VVG-AJ0', 'VVG-NN1', 'VVZ-NN2',
    'AJ0', 'AJC', 'AJS',
    'AJ0-NN1', 'AJ0-VVG', 'AJ0-VVN',
    'AV0', 'AVQ', 'AVP',
    'AV0-AJ0'
])

In [180]:
content_words = all_words[all_words['word_type'].isin(content_word_pos)]
display(len(content_words))
content_words.head(10)

25360

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic
6,25076,KAHLO,kahlo,NP0,,,1267,a6u-fragment02,a6u,academic
7,25077,CULT,cult,NN1,,,1267,a6u-fragment02,a6u,academic
8,25078,Oriana,oriana,NP0-NN1,,,1268,a6u-fragment02,a6u,academic
9,25079,Baddeley,baddeley,NP0,,,1268,a6u-fragment02,a6u,academic
13,25083,witnessed,witness,VVN,mrw,met,1269,a6u-fragment02,a6u,academic
15,25085,shift,shift,NN1-VVB,,,1269,a6u-fragment02,a6u,academic


## Sparsification

Create a vocabulary using content words. This will be used as the unigram features.

In [181]:
vocabulary = content_words[['word']].drop_duplicates().set_index('word')

display(len(vocabulary))
vocabulary.sample(10)

7327

""
word
too
beforehand
chronic
Kates
Terney
conference
Economic
incorrigible
Jackson


In [182]:
def sparse_featurize(words, vocabulary):
    print('Build iterable sentences.')
    iterable_sentences = words.groupby('sentence_id')['word'].apply(list)
    display(len(iterable_sentences))
    display(iterable_sentences[:5])

    indices = []
    data = []
    indptr = [0]
    for i, word, sentence_id in words[['word', 'sentence_id']].itertuples():
        sentence = iterable_sentences[sentence_id]

        # Used to generate a row of 0's and 1's for sparse matrix
        unique_unigram_indices = set()
        for context_word in sentence:
            if context_word == word:
                continue
            if context_word not in vocabulary.index:
                unique_unigram_indices.add(len(vocabulary))
            unique_unigram_indices.add(vocabulary.index.get_loc(context_word))

        # Data = 1 at column index of sparse matrix = row index of word in vocabulary.
        indices.extend(unique_unigram_indices)  # Indices of the data values in this row.
        data.extend(np.ones(len(unique_unigram_indices), dtype=int))  # Data of all one's.
        indptr.append(indptr[-1]+len(unique_unigram_indices))

        if not i % 2000:
            print('-', end='', flush=True)

    return (
        csr_matrix((data, indices, indptr), dtype=int, shape=(len(words), len(vocabulary)+1)),
        iterable_sentences
    )

Width of sparse matrix is len(vocabulary)+1 to process unknown words.

In [183]:
words = content_words
sparse_unigram_features, iterable_sentences = sparse_featurize(words, vocabulary)

scipy.sparse.save_npz('data/sparse_unigram_features.npz', sparse_unigram_features)
display(sparse_unigram_features)
sparse_unigram_features.shape

Build iterable sentences.


1961

sentence_id
1267    [DRESS, HANGS, HERE', DE-FROCKING, KAHLO, CULT]                                                                               
1268    [Oriana, Baddeley]                                                                                                            
1269    [witnessed, shift, art, establishment, attitudes, art, produced, traditional, parameters]                                     
1270    [work, previously, marginalised, artists, become, area, rich, speculation, art, dealers, priced, modern, masters, market]     
1271    [Almost, year, witnessed, discovery, new, artistic, terrain, graffiti, art, Soviet, art, Australian, art, art, Latin, America]
Name: word, dtype: object

-----------

<25360x7328 sparse matrix of type '<class 'numpy.int32'>'
	with 386398 stored elements in Compressed Sparse Row format>

(25360, 7328)

Inspect if sparse features are valid.

In [184]:
sparse_unigram_features = scipy.sparse.load_npz('data/sparse_unigram_features.npz')

In [185]:
select_word_i = 210

selected_word = words.iloc[select_word_i]
display(selected_word)
sentence_of_selected_word = sorted(list(set(iterable_sentences.loc[selected_word.sentence_id])))
display(sentence_of_selected_word)

unigram_select_indices = np.nonzero(sparse_unigram_features[select_word_i])[:50][1]  # Nonzero indices indicating word.
display(unigram_select_indices)
sparse_matrix_sentence_selections = sorted(list(vocabulary.iloc[unigram_select_indices].index))
display(sparse_matrix_sentence_selections)

assert len(sentence_of_selected_word) - 1 == len(sparse_matrix_sentence_selections)

word_id        25449         
word           passionate    
lemma          passionate    
word_type      AJ0           
function                     
seg_type                     
sentence_id    1282          
text_id        a6u-fragment02
text_tag       a6u           
genre          academic      
Name: 379, dtype: object

['Diego',
 'Rivera',
 'appearance',
 'come',
 'dominate',
 'emotional',
 'flamboyant',
 'husband',
 'obsession',
 'pain',
 'passionate',
 'physical',
 'responses',
 'work']

array([ 70,  71, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,  16])

['Diego',
 'Rivera',
 'appearance',
 'come',
 'dominate',
 'emotional',
 'flamboyant',
 'husband',
 'obsession',
 'pain',
 'physical',
 'responses',
 'work']

Looks good!

Do you wanna build a model?

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.estimator_checks import check_estimator

def preprocess_data(words):
    def build_content_words(all_words):
        print('Building content words.')
        # Keeping content words: http://ucrel.lancs.ac.uk/bnc2/bnc2guide.htm#m2adv
        # Does not include forms of 'be', 'do', 'have', or modal verbs,
        # as suggested in https://github.com/EducationalTestingService/metaphor/tree/master/content-words
        content_word_pos = set([
            'NN1', 'NN2', 'NN0',
            'NP0',
            'NN1-NP0', 'NN1-VVB', 'NN1-VVG', 'NN2-VVZ',
            'NP0-NN1',
            'VVB', 'VVD', 'VVG', 'VVI', 'VVN', 'VVZ',
            'VVB-NN1', 'VVD-VVN', 'VVD-AJ0', 'VVG-AJ0', 'VVG-NN1', 'VVZ-NN2',
            'AJ0', 'AJC', 'AJS',
            'AJ0-NN1', 'AJ0-VVG', 'AJ0-VVN',
            'AV0', 'AVQ', 'AVP',
            'AV0-AJ0'
        ])
        
        content_words = all_words[all_words['word_type'].isin(content_word_pos)]
        
        return content_words
    
    def get_test_fold_ids(words):
        num_folds = {
            'newspapers': 10,
            'conversation': 10,
            'academic': 10,
            'fiction': 11
        }
        
        genre = words.iloc[0]['genre']

        test_fold_ids = defaultdict(int)

        for i, (text, _) in enumerate(words.groupby('text_id')):
            test_fold_ids[text] = i // num_folds[genre]

        return words['text_id'].apply(lambda x: test_fold_ids[x])
        
    
    def build_labels(words):
        labels = words['function'] == 'mrw'
        
        return labels
    
    content_words = build_content_words(words)
    test_fold_ids = get_test_fold_ids(content_words)
    labels = build_labels(content_words)
    
    assert len(content_words) == len(test_fold_ids)
    
    return content_words, test_fold_ids, labels

Model

In [211]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.estimator_checks import check_estimator

def preprocess_data(words):
    def build_content_words(all_words):
        print('Building content words.')
        # Keeping content words: http://ucrel.lancs.ac.uk/bnc2/bnc2guide.htm#m2adv
        # Does not include forms of 'be', 'do', 'have', or modal verbs,
        # as suggested in https://github.com/EducationalTestingService/metaphor/tree/master/content-words
        content_word_pos = set([
            'NN1', 'NN2', 'NN0',
            'NP0',
            'NN1-NP0', 'NN1-VVB', 'NN1-VVG', 'NN2-VVZ',
            'NP0-NN1',
            'VVB', 'VVD', 'VVG', 'VVI', 'VVN', 'VVZ',
            'VVB-NN1', 'VVD-VVN', 'VVD-AJ0', 'VVG-AJ0', 'VVG-NN1', 'VVZ-NN2',
            'AJ0', 'AJC', 'AJS',
            'AJ0-NN1', 'AJ0-VVG', 'AJ0-VVN',
            'AV0', 'AVQ', 'AVP',
            'AV0-AJ0'
        ])
        
        content_words = all_words[all_words['word_type'].isin(content_word_pos)]
        
        return content_words
    
    def get_test_fold_ids(words):
        num_folds = {
            'newspapers': 10,
            'conversation': 10,
            'academic': 10,
            'fiction': 11
        }
        
        genre = words.iloc[0]['genre']

        test_fold_ids = defaultdict(int)

        for i, (text, _) in enumerate(words.groupby('text_id')):
            test_fold_ids[text] = i // num_folds[genre]

        return words['text_id'].apply(lambda x: test_fold_ids[x])
        
    
    def build_labels(words):
        labels = words['function'] == 'mrw'
        
        return labels
    
    content_words = build_content_words(words)
    test_fold_ids = get_test_fold_ids(content_words)
    labels = build_labels(content_words)
    
    assert len(content_words) == len(test_fold_ids)
    
    return content_words, test_fold_ids, labels

class UnigramClassifier(BaseEstimator):
    def __init__(self, verbose=10, random_state=0, C=10, penalty='l1',
                 solver='liblinear', class_weight='balanced', max_iter=500):
        self.verbose = verbose
        self.random_state = random_state
        self.C = C
        self.penalty = penalty
        self.solver = solver
        self.class_weight = class_weight
        self.max_iter = max_iter
 
    
    def fit(self, X, y):
        self.build_features(X, y)
        
        self.clf_ = LogisticRegression(
            verbose=self.verbose,
            random_state=self.random_state,
            C=self.C,
            penalty=self.penalty,
            solver=self.solver,
            class_weight=self.class_weight,
            max_iter=self.max_iter
        )
        
        self.clf_.fit(self.X_, self.y_)
        
        return self
    
    def build_features(self, X, y):
        self.vocabulary_ = X[['word']].drop_duplicates().set_index('word')
        
        sparse_unigram_features, iterable_sentences = self.sparse_featurize(X)
        
        self.X_, self.y_ = shuffle(sparse_unigram_features, y, random_state=0)
    
    
    def sparse_featurize(self, words):
        iterable_sentences = words.groupby('sentence_id')['word'].apply(list)

        indices = []
        data = []
        indptr = [0]
        for i, word, sentence_id in words[['word', 'sentence_id']].itertuples():
            sentence = iterable_sentences[sentence_id]

            # Used to generate a row of 0's and 1's for sparse matrix
            unique_unigram_indices = set()
            for context_word in sentence:
                if context_word == word:
                    continue
                if context_word not in self.vocabulary_.index:
                    unique_unigram_indices.add(len(self.vocabulary_))
                else:
                    unique_unigram_indices.add(self.vocabulary_.index.get_loc(context_word))

            # Data = 1 at column index of sparse matrix = row index of word in vocabulary.
            indices.extend(unique_unigram_indices)  # Indices of the data values in this row.
            data.extend(np.ones(len(unique_unigram_indices), dtype=int))  # Data of all one's.
            indptr.append(indptr[-1]+len(unique_unigram_indices))

            if not i % 2000:
                print('-', end='', flush=True)

        return (
            csr_matrix((data, indices, indptr), dtype=int, shape=(len(words), len(self.vocabulary_)+1)),
            iterable_sentences
        )   
    
    
    def predict(self, X):
        featurized_X = self.sparse_featurize(X)[0]
        return self.clf_.predict(featurized_X)
    
# check_estimator(UnigramClassifier)

In [303]:
%run model_functions.ipynb

Try on academic.

In [304]:
X, test_fold_ids, vocabulary, y = preprocess_data(all_words)

-----------

In [305]:
unigram_classifier = UnigramClassifier(
    verbose=10, random_state=0, C=10, penalty='l1',
    solver='liblinear', class_weight='balanced', max_iter=500,
    vocabulary=vocabulary
)

C = np.logspace(-2,4,20)
penalty = ['l1', 'l2']
hyperparameters = {'C':C, 'penalty':penalty}
clf = GridSearchCV(unigram_classifier, hyperparameters,
                   cv=PredefinedSplit(test_fold_ids),
                   verbose=10, scoring='f1')

In [ ]:
clf.fit(X, y)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[CV] C=0.01, penalty=l1 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 17 epochs took 0 seconds
[CV] ..... C=0.01, penalty=l1, score=0.3702857142857143, total=   0.3s
[CV] C=0.01, penalty=l1 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 18 epochs took 0 seconds
[CV] ..... C=0.01, penalty=l1, score=0.3415032679738562, total=   0.3s
[CV] C=0.01, penalty=l1 ..............................................


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 155 seconds
[CV]  C=1.623776739188721, penalty=l1, score=0.24350205198358416, total= 2.6min
[CV] C=1.623776739188721, penalty=l1 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 180 seconds
[CV]  C=1.623776739188721, penalty=l1, score=0.16334661354581673, total= 3.0min
[CV] C=1.623776739188721, penalty=l1 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [193]:
clf.best_params_, clf.best_estimator_

({'C': 0.04281332398719394, 'penalty': 'l1'},
 UnigramClassifier(C=0.04281332398719394, class_weight='balanced',
          max_iter=500, penalty='l1', random_state=0, solver='liblinear',
          verbose=10))

In [194]:
test_set = pd.read_csv('data/test/academic/words.csv', encoding='ISO--8859-1', na_filter=False)
display(len(test_set))
test_set.head()

17223

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,61516,BRINGING,bring,VVG,mrw,met,3232,amm-fragment02,amm,academic
1,61517,FOSSILS,fossil,NN2,,,3232,amm-fragment02,amm,academic
2,61518,BACK,back,AVP,mrw,met,3232,amm-fragment02,amm,academic
3,61519,TO,to,PRP,mrw,met,3232,amm-fragment02,amm,academic
4,61520,LIFE,life,NN1,mrw,met,3232,amm-fragment02,amm,academic


In [196]:
X_test, _, y_test = preprocess_data(test_set)

preds = clf.predict(X_test)

print(classification_report(y_test, preds))

Building content words.
----              precision    recall  f1-score   support

       False       0.77      0.23      0.36      6744
        True       0.22      0.76      0.34      1882

   micro avg       0.35      0.35      0.35      8626
   macro avg       0.49      0.49      0.35      8626
weighted avg       0.65      0.35      0.35      8626



Let's try our model on all 4 genres!

In [40]:
results = {
    'academic':None,
    'fiction':None,
    'conversation':None,
    'newspapers':None
}

with open('results/context_unigram_model2.txt', 'w') as f:
    for genre in results:
        try:
            train_set = pd.read_csv('data/train/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
            X_train, test_fold_ids, vocabulary, y_train = preprocess_data(train_set)
            
            unigram_classifier = UnigramClassifier(
                verbose=10, random_state=0, C=10, penalty='l1',
                solver='saga', class_weight='balanced', max_iter=500,
                vocabulary=vocabulary
            )

            C = np.logspace(-2,4,20)
            penalty = ['l1', 'l2']
            hyperparameters = {'C':C, 'penalty':penalty}
            clf = GridSearchCV(unigram_classifier, hyperparameters,
                               cv=PredefinedSplit(test_fold_ids),
                               verbose=10, scoring='f1')
            
            clf.fit(X_train, y_train)

            test_set = pd.read_csv('data/test/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
            X_test, _, _, y_test = preprocess_data(test_set)
            preds = clf.predict(X_test)

            result = classification_report(y_test, preds, output_dict)
            print(result)
            results[genre] = result
        except:
            print(results)
            f.write(json.dumps(results))

Building content words.
Fitting 10 folds for each of 40 candidates, totalling 400 fits
[CV] C=0.01, penalty=l1 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-----------[LibLinear]---------{'academic': None, 'fiction': None, 'conversation': None, 'newspapers': None}
Building content words.
Fitting 10 folds for each of 40 candidates, totalling 400 fits
[CV] C=0.01, penalty=l1 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


----------[LibLinear]-----------[CV] .................... C=0.01, penalty=l1, score=0.0, total=   2.1s
[CV] C=0.01, penalty=l1 ..............................................


...

-----[LibLinear]------[CV] .................... C=0.01, penalty=l1, score=0.0, total=   1.4s
[CV] C=0.01, penalty=l1 ..............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   31.4s remaining:    0.0s


-----[LibLinear]------[CV] .................... C=0.01, penalty=l1, score=0.0, total=   1.8s
[CV] C=0.01, penalty=l2 ..............................................
-----[LibLinear]------[CV] .... C=0.01, penalty=l2, score=0.18181818181818182, total=   1.8s
[CV] C=0.01, penalty=l2 ..............................................
------[LibLinear]------[CV] .... C=0.01, penalty=l2, score=0.23434991974317818, total=   1.8s
[CV] C=0.01, penalty=l2 ..............................................
-----[LibLinear]------[CV] ..... C=0.01, penalty=l2, score=0.1669024045261669, total=   1.5s
[CV] C=0.01, penalty=l2 ..............................................
-----[LibLinear]------[CV] .... C=0.01, penalty=l2, score=0.22752808988764048, total=   1.5s
[CV] C=0.01, penalty=l2 ..............................................
{'academic': None, 'fiction': None, 'conversation': None, 'newspapers': None}
Building content words.
Fitting 10 folds for each of 40 candidates, totalling 400 fits
[CV] C=0.01, p

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


----{'academic': None, 'fiction': None, 'conversation': None, 'newspapers': None}


In [32]:
results

{'academic': '              precision    recall  f1-score   support\n\n       False       0.77      0.23      0.36      6744\n        True       0.22      0.76      0.34      1882\n\n   micro avg       0.35      0.35      0.35      8626\n   macro avg       0.49      0.49      0.35      8626\nweighted avg       0.65      0.35      0.35      8626\n',
 'fiction': '              precision    recall  f1-score   support\n\n       False       0.85      0.76      0.80      2837\n        True       0.18      0.28      0.22       524\n\n   micro avg       0.68      0.68      0.68      3361\n   macro avg       0.51      0.52      0.51      3361\nweighted avg       0.75      0.68      0.71      3361\n',
 'conversation': '              precision    recall  f1-score   support\n\n       False       0.90      0.75      0.82      2368\n        True       0.13      0.33      0.19       282\n\n   micro avg       0.70      0.70      0.70      2650\n   macro avg       0.52      0.54      0.50      2650\nwe

## Other

In [ ]:
# scoring = ['accuracy', 'precision', 'recall', 'f1']
# scores = cross_validate(clf, scoring=scoring, X=X, y=y, cv=10, verbose=10)
# scores, sum(scores['test_f1']/len(scores['test_f1']))

In [ ]:
# clf = LogisticRegression(verbose=10, random_state=0, C=10, penalty='l1',
#                          solver='liblinear', class_weight='balanced', max_iter=500)

In [ ]:
# hyperparameters = {'C':[1, 5, 10, 20, 100], 'penalty':['l1', 'l2']}
# from sklearn.model_selection import GridSearchCV
# clf = GridSearchCV(LogisticRegression(class_weight='balanced', verbose=10), hyperparameters,
#                                       verbose=10, cv=10, scoring='f1')
# clf.fit(X_train, y_train)